<a href="https://colab.research.google.com/github/jhenioliver/santander-dev-week-etl/blob/main/SantanderDevWeek2023_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**ID's dos usuários:**

Usuários criados através da API Santander Dew Week 2023.

  ```
  UserID
  5196
  5197
  5198
  ```

In [2]:
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

In [5]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[5196, 5197, 5198]


In [6]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 5196,
    "name": "Jen",
    "account": {
      "id": 5523,
      "number": "10.123456-7",
      "agency": "2037",
      "balance": 627.12,
      "limit": 1500.0
    },
    "card": {
      "id": 5055,
      "number": "xxxx xxxx 1232 1811",
      "limit": 2500.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 5197,
    "name": "Jin",
    "account": {
      "id": 5524,
      "number": "10.654456-9",
      "agency": "2837",
      "balance": 927.12,
      "limit": 1500.0
    },
    "card": {
      "id": 5056,
      "number": "xxxx xxxx 1782 1811",
      "limit": 2500.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 5198,
    "name": "John",
    "account": {
      "id": 5525,
      "number": "10.78945-5",
      "agency": "2787",
      "balance": 987.12,
      "limit": 1500.0
    },
    "card": {
      "id": 5057,
      "number": "xxxx xxxx 1782 1878",
      "limit": 2500.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.3 MB/s eta 0:00:00


In [ ]:
openai_api_key = '********************'

In [ ]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância e benefício dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [7]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Jen updated? True!
User Jin updated? True!
User John updated? True!
